In [ ]:
import xarray as xr
import math
from pyproj import CRS, Transformer
import pyproj
import numpy as np
import xesmf as xe
import matplotlib.pyplot as plt
import fsspec 
from dask.distributed import Client, progress
import re
from cfgrib import xarray_to_grib
import pickle
import gc

Global Parameters

In [ ]:
CMP_MAX = 1000.0
BUCKET_NAME = 'noaa-nws-hafs-pds'
DROP_VARIABLES = ['q', 'w', 'wz', 'absv', 'clwmr', 'icmr', 'rwmr', 'snmr','grle', 'rare', 'dpt']



In [ ]:
try:
    from dask.distributed import get_client
    get_client().close()
except Exception:
    pass

client = Client()  # set up local cluster on your laptop
client

fs = fsspec.filesystem('s3', anon=True)

Project Functions

In [ ]:
def frame_processing(s3_path: str):
    '''
    Docstring for frame_processing
    
    :param ds: A HAFSA model timestep
    :type ds: xr.Dataset

    Checks for frame validity:
        No land within 200km of center
        Central minimum pressure of at least 990mb
    
    If valid returns a Model Dataset consisting of:
        PBL temp, heights, rh, and radial/azimuthal winds
        700 mb temp, heights, rh, and radial/azimuthal winds
        central minimum pressure for the frame,
        maximum wind for the frame
    '''
    header_path = 'simplecache::s3://noaa-nws-hafs-pds/'

    frame_path = header_path + s3_path
    nxt_frame_path = header_path = re.sub('.f024', '.f027', frame_path)

    frame_file = fsspec.open_local(frame_path, s3={'anon': True}, filecache={'cache_storage':'/tmp/files'})
    nxt_frame_file = fsspec.open_local(nxt_frame_path, s3={'anon': True}, filecache={'cache_storage':'/tmp/files'})

    
    ds_sfc = xr.open_dataset(frame_file,  
                         filter_by_keys={'typeOfLevel': 'meanSea'},
                         engine = 'cfgrib')
    
    ds_sfc = ds_sfc['prmsl'].compute()/100
    c_mslp = ds_sfc.min()
    

    if c_mslp > CMP_MAX:
        return None
    else:
        
        center_coords = ds_sfc.where(ds_sfc == c_mslp, drop=True).squeeze()

    ds_atm = xr.open_dataset(frame_file,
                         drop_variables = DROP_VARIABLES, 
                         filter_by_keys={'typeOfLevel': 'isobaricInhPa'},
                         engine = 'cfgrib'
                        )   
    
    ds_sfc_nxt_step = xr.open_dataset(nxt_frame_file, 
                         filter_by_keys={'typeOfLevel': 'meanSea'},
                         engine = 'cfgrib')
    
    ds_atm = ds_atm.sel(isobaricInhPa = [c_mslp.values, 700.0], method = 'nearest')
    
    
    # computing the necessary values for the follow on frame. Necessary
    # for determining storm motion

    ds_sfc_nxt_step = ds_sfc_nxt_step['prmsl'].compute()
    c_mslp_nxt_step = ds_sfc_nxt_step.min()
    center_coords_nxt_step = ds_sfc_nxt_step.where(ds_sfc_nxt_step == c_mslp_nxt_step, drop=True).squeeze()

    storm_heading = calc_heading(center_coords.latitude.values, center_coords.longitude.values, 
                                 center_coords_nxt_step.latitude.values, center_coords_nxt_step.longitude.values)

    frame = to_polar(ds_atm.compute(),
                     origin_lat= center_coords.latitude.values,
                     origin_lon= center_coords.longitude.values,
                     storm_heading = storm_heading)
    
    
    atm_object = {
                    'radius_coords': frame.radius.values,
                    'angle_coords': frame.angle.values,
                    'gh': frame['gh'].isel(isobaricInhPa = 0).values,
                    't': frame['t'].isel(isobaricInhPa = 0).values,
                    'r': frame['r'].isel(isobaricInhPa = 0).values,
                    'u': frame['u'].isel(isobaricInhPa = 0).values,
                    'v': frame['v'].isel(isobaricInhPa = 0).values,
                    'center_lat': center_coords.latitude.values,
                    'center_lon': center_coords.longitude.values,
                    'valid_time': frame.isel(isobaricInhPa = 0).valid_time.values,
                    'heading': storm_heading
                    }
    sfc_object = {
                    'radius_coords': frame.radius.values,
                    'angle_coords': frame.angle.values,
                    'gh': frame['gh'].isel(isobaricInhPa = 1).values,
                    't': frame['t'].isel(isobaricInhPa = 1).values,
                    'r': frame['r'].isel(isobaricInhPa = 1).values,
                    'u': frame['u'].isel(isobaricInhPa = 1).values,
                    'v': frame['v'].isel(isobaricInhPa = 1).values,
                    'center_pressure': c_mslp.values,
                    'center_lat': center_coords.latitude.values,
                    'center_lon': center_coords.longitude.values,
                    'valid_time': frame.valid_time.values,
                    'heading': storm_heading
                    }
                    
    
    return sfc_object, atm_object
    
def calc_heading(lat1, lon1, lat2, lon2):

    lat1_rad = np.deg2rad(lat1)
    lat2_rad = np.deg2rad(lat2)
    lon1_rad = np.deg2rad(lon1)
    lon2_rad = np.deg2rad(lon2)

    d_lon = lon2_rad - lon1_rad

    y = np.sin(d_lon) * np.cos(lat2_rad)
    x = (np.cos(lat1_rad) * np.sin(lat2_rad) -
         np.sin(lat1_rad) * np.cos(lat2_rad) * np.cos(d_lon)) 
    
    initial_bearing_rad = np.arctan2(y, x)

    initial_bearing_deg = np.rad2deg(initial_bearing_rad)

    bearing = (initial_bearing_deg + 360) % 360

    return bearing 

def rotate_point_around_origin(x, y, angle):
    """
    Rotates a 2D point (x, y) around the origin (0, 0) by a given angle.

    Args:
        x (float): The x-coordinate of the point.
        y (float): The y-coordinate of the point.
        angle_radians (float): The angle of rotation in radians (counter-clockwise).

    Returns:
        tuple: A tuple (new_x, new_y) representing the rotated point.
    """
    angle = np.deg2rad(angle)
    new_x = x * math.cos(angle) - y * math.sin(angle)
    new_y = x * math.sin(angle) + y * math.cos(angle)
    return new_x, new_y

def to_polar(
        ds: xr.Dataset,
        origin_lat: float,
        origin_lon: float,
        storm_heading: float,
        R_max = 250.0, # Max range in km
        R_step = 0.5, # Range step in km
        A_step = 0.5  # Azimuth step in degrees
    ) -> xr.Dataset:
    
        R = 6378137 # radius of Earth in meters

        new_range = np.arange(0.0, R_max + R_step, R_step)
        new_azimuth = np.arange(0.0, 360.0, A_step)
        R, T = np.meshgrid(new_azimuth, new_range, indexing = 'ij')
     
        X = R * np.cos(T) * 1000 # Convert km to meters
        Y = R * np.sin(T) * 1000
        local_proj_str = f"+proj=aeqd +lat_0={origin_lat} +lon_0={origin_lon} +units=m"
        transformer = pyproj.Transformer.from_crs(local_proj_str, "EPSG:4326", always_xy=True)

        X,Y = rotate_point_around_origin(X, Y, storm_heading)
        target_lons, target_lats = transformer.transform(X, Y)
        
        # print("Target Lats: " + str(np.shape(target_lats)))
        # print("Target Lons: " + str(np.shape(target_lons)))
        # print("range: " + str(np.shape(new_range)))
        # print("azimuth: " + str(np.shape(new_azimuth)))

        # print("New Range: " + str(new_range.max()))
        # print("New Angle: " + str(new_azimuth.max()))
        ds_out = xr.Dataset(
                            coords={
                                    "latitude": (("angle", "radius"), target_lats),
                                    "longitude": (("angle", "radius"), target_lons),
                                    "radius": new_range,
                                    "angle": new_azimuth
                                        }
                            )
        regridder = xe.Regridder(ds, ds_out, 'bilinear')
        polar_out = regridder(ds)
       
        return polar_out


         


In [ ]:
loaded_list = []

with open('Data/links/link_list.txt', 'r') as f:
            for line in f:
                loaded_list.append(line.strip())

In [ ]:
starting_frame = 400
partial_loaded_list = loaded_list[starting_frame:]

In [ ]:
frames_700mb = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'u': [],
                    'v': [],
                    'valid_time': []

    }
frames_sfc = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'u': [],
                    'v': [],
                    'center_pressure': [],
                    'valid_time': []

    }
try:
    sfc_frame, atm_frame = frame_processing(partial_loaded_list[0])
    frame = True
    start = '0'
except:
    print("Issue with link {partial_loaded_list[0]}")
    frame = False
if frame:

    frames_700mb['radius_coords'].append(atm_frame['radius_coords'])
    frames_700mb['angle_coords'].append(atm_frame['angle_coords'])
    frames_700mb['gh'].append(atm_frame['gh'])
    frames_700mb['t'].append(atm_frame['t'])
    frames_700mb['r'].append(atm_frame['r'])
    frames_700mb['u'].append(atm_frame['u'])
    frames_700mb['v'].append(atm_frame['v'])
    frames_700mb['valid_time'].append(atm_frame['valid_time'])

    frames_sfc['radius_coords'].append(sfc_frame['radius_coords'])
    frames_sfc['angle_coords'].append(sfc_frame['angle_coords'])
    frames_sfc['gh'].append(sfc_frame['gh'])
    frames_sfc['t'].append(sfc_frame['t'])
    frames_sfc['r'].append(sfc_frame['r'])
    frames_sfc['u'].append(sfc_frame['u'])
    frames_sfc['v'].append(sfc_frame['v'])
    frames_sfc['valid_time'].append(sfc_frame['valid_time'])
for i, frame_path in enumerate(partial_loaded_list[1:4]):
    print(i)
    try:
        sfc_frame, atm_frame = frame_processing(frame_path)
        frame = True
    except:
        print("Issue with link {}", frame_path)
        frame = False
    if frame:
        
        frames_700mb['radius_coords'].append(atm_frame['radius_coords'])
        frames_700mb['angle_coords'].append(atm_frame['angle_coords'])
        frames_700mb['gh'].append(atm_frame['gh'])
        frames_700mb['t'].append(atm_frame['t'])
        frames_700mb['r'].append(atm_frame['r'])
        frames_700mb['u'].append(atm_frame['u'])
        frames_700mb['v'].append(atm_frame['v'])
        frames_700mb['valid_time'].append(atm_frame['valid_time'])

        frames_sfc['radius_coords'].append(sfc_frame['radius_coords'])
        frames_sfc['angle_coords'].append(sfc_frame['angle_coords'])
        frames_sfc['gh'].append(sfc_frame['gh'])
        frames_sfc['t'].append(sfc_frame['t'])
        frames_sfc['r'].append(sfc_frame['r'])
        frames_sfc['u'].append(sfc_frame['u'])
        frames_sfc['v'].append(sfc_frame['v'])
        frames_sfc['valid_time'].append(sfc_frame['valid_time'])
       
    if i%10 == 0:
        end = str(starting_frame + i)
        # with open("CompleteData/all_frames_" + str(start) + "to" + str(end) + ".pkl", "wb") as f:
        #     pickle.dump(all_frames, f)
        
        ds = xr.Dataset(
                        data_vars = {
                                     'gh': (['levels','valid_time', 'angle', 'radius' ], np.stack([frames_sfc['gh'], frames_700mb['gh']])),
                                     'rh': (['levels','valid_time','angle', 'radius'], np.stack([frames_sfc['r'], frames_700mb['r']])),
                                     't': (['levels','valid_time', 'angle', 'radius'], np.stack([frames_sfc['t'], frames_700mb['t']])),
                                     'u': (['levels','valid_time', 'angle', 'radius'], np.stack([frames_sfc['u'], frames_700mb['u']])),
                                     'v': (['levels','valid_time', 'angle', 'radius'], np.stack([frames_sfc['v'], frames_700mb['v']])),
                                    },
                        coords= {
                                 'angle': (frames_sfc['angle_coords'][0]), 
                                 'radius': (frames_sfc['radius_coords'][0]), 
                                 'valid_times': frames_sfc['valid_time'],
                                 'levels': (['sfc', 700])
                                }
        )
        ds.to_netcdf('test.nc')
        del fram
        start = i + starting_frame + 1
        all_frames = {
                    'radius_coords': [],
                    'angle_coords': [],
                    'gh': [],
                    't': [],
                    'r': [],
                    'center_pressure': [],
                    'valid_time': []
                     }
    del frame
    gc.collect()    



skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "c:\Users\12ian\anaconda3\envs\ATMS523-Proj-Everything-Else\Lib\site-packages\cfgrib\dataset.py", line 726, in build_dataset_components
    dict_merge(variables, coord_vars)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\12ian\anaconda3\envs\ATMS523-Proj-Everything-Else\Lib\site-packages\cfgrib\dataset.py", line 642, in dict_merge
    raise DatasetBuildError(
    ...<2 lines>...
    )
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variab

965.955
965.955


skipping variable: paramId==3017 shortName='dpt'
Traceback (most recent call last):
  File "c:\Users\12ian\anaconda3\envs\ATMS523-Proj-Everything-Else\Lib\site-packages\cfgrib\dataset.py", line 726, in build_dataset_components
    dict_merge(variables, coord_vars)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\12ian\anaconda3\envs\ATMS523-Proj-Everything-Else\Lib\site-packages\cfgrib\dataset.py", line 642, in dict_merge
    raise DatasetBuildError(
    ...<2 lines>...
    )
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  825.,  800.,
        775.,  750.,  725.,  700.,  675.,  650.,  625.,  600.,  575.,
        550.,  525.,  500.,  475.,  450.,  425.,  400.,  375.,  350.,
        325.,  300.,  275.,  250.,  225.,  200.,  175.,  150.,  125.,
        100.,   70.,   50.,   30.,   20.,   10.,    7.,    5.,    2.])) new_value=Variab

0
946.308
946.308


NameError: name 'all_frames' is not defined

In [43]:
np.stack([frames_sfc['gh'], frames_700mb['gh']]).shape

(2, 4, 720, 501)

In [49]:
ds['gh'].sel(levels = 'sfc')

<xarray.DataArray 'gh' (valid_time: 2, angle: 720, radius: 501)> Size: 3MB
array([[[2793.0535, 2793.0535, 2793.0535, ..., 2793.0535, 2793.0535,
         2793.0535],
        [2793.017 , 2793.2004, 2793.47  , ..., 2793.019 , 2792.8572,
         2792.7458],
        [2792.9822, 2793.3477, 2793.8772, ..., 2792.981 , 2792.6533,
         2792.4338],
        ...,
        [3069.3901, 3085.8   , 3090.1096, ..., 3103.861 , 3104.5513,
         3098.205 ],
        [3069.751 , 3086.3523, 3090.3035, ..., 3103.9282, 3104.571 ,
         3098.28  ],
        [3070.3645, 3086.7102, 3090.4932, ..., 3103.9905, 3104.5955,
         3098.3557]],

       [[2625.9482, 2625.9482, 2625.9482, ..., 2625.9482, 2625.9482,
         2625.9482],
        [2625.5266, 2625.5452, 2625.6414, ..., 2626.7546, 2626.5657,
         2626.2974],
        [2625.1096, 2625.2046, 2625.3975, ..., 2627.528 , 2627.1687,
         2626.6465],
        ...,
        [3143.7888, 3143.0366, 3146.5056, ..., 3159.0256, 3152.7524,
         3148.2505],
        [3143.785 , 3143.0217, 3146.495 , ..., 3159.0476, 3152.758 ,
         3148.2673],
        [3143.7812, 3143.0085, 3146.4849, ..., 3159.0725, 3152.7625,
         3148.2844]]], shape=(2, 720, 501), dtype=float32)
Coordinates:
  * angle    (angle) float64 6kB 0.0 0.5 1.0 1.5 2.0 ... 358.0 358.5 359.0 359.5
  * radius   (radius) float64 4kB 0.0 0.5 1.0 1.5 ... 248.5 249.0 249.5 250.0
    levels   <U21 84B 'sfc'
Dimensions without coordinates: valid_time

In [35]:
frames_700mb['gh']

[array([[-82.867096, -82.867096, -82.867096, ..., -82.867096, -82.867096,
         -82.867096],
        [-82.81316 , -82.82051 , -82.79453 , ..., -82.79585 , -82.79815 ,
         -82.818375],
        [-82.75923 , -82.773926, -82.71293 , ..., -82.723045, -82.7261  ,
         -82.76786 ],
        ...,
        [217.15504 , 239.85675 , 247.19435 , ..., 257.39285 , 253.04007 ,
         247.57262 ],
        [216.4593  , 239.98303 , 247.24992 , ..., 257.52    , 253.18625 ,
         247.65155 ],
        [215.91356 , 240.01015 , 247.29913 , ..., 257.66785 , 253.33096 ,
         247.72916 ]], shape=(720, 501), dtype=float32),
 [...]]